In [1]:
import numpy as np
from mesh.mesh_read_plot3D import StructuredMeshInitialization2D
from mesh.mesh import MeshGeoCalculator2D
import boundary.boundary as bd
import type_transform as tf
import config
import Initialization as initial

# 读取并预处理网格

In [2]:
mesh_read = StructuredMeshInitialization2D()
mesh_read.load_file("airfoil0012extend.grd", "airfoil0012extend.inp", 0.001)
mesh_read.merge_blocks_2D()
mesh_read.interface_transform_cal()
mesh_read.mesh_reader.info()
mesh_read.bc_reader.info()
mesh_read.print_block_info()

[Block 0] shape: (259, 49, 1)
  x range: -14.898221 ~ 15.908266
  y range: -15.695282 ~ 15.697121
  z range: 0.000000 ~ 0.000000
[Block 1] shape: (10, 49, 1)
  x range: 1.000000 ~ 15.908266
  y range: -0.163128 ~ 0.215341
  z range: 0.000000 ~ 0.000000
Detected 2D boundary condition file.
[Block 0] A shape: (259, 49, 1)
  - Source: (259, 130, 1, 1), Type: 2
  - Source: (259, 259, 1, 49), Type: -1, Target: (1, 1, 1, 49), Block: 1
  - Source: (130, 259, 49, 49), Type: 4
  - Source: (130, 1, 1, 1), Type: 2
  - Source: (1, 130, 49, 49), Type: 4
  - Source: (1, 1, 1, 49), Type: -1, Target: (10, 10, 1, 49), Block: 1
[Block 1] B shape: (10, 49, 1)
  - Source: (10, 1, 1, 1), Type: 2
  - Source: (10, 10, 1, 49), Type: -1, Target: (1, 1, 1, 49), Block: 0
  - Source: (1, 10, 49, 49), Type: 4
  - Source: (1, 1, 1, 49), Type: -1, Target: (259, 259, 1, 49), Block: 0
[Block 0]  shape: (259, 49, 1)
  - BC type 2, source (259, 1, 1, 1), target_block N/A, target N/A
  - BC type -1, source (259, 259, 1, 

# 计算网格几何参数

In [3]:
mesh_geocal = MeshGeoCalculator2D(mesh_read)
mesh_geocal.compute_centroids()
mesh_geocal.compute_volumes()
mesh_geocal.compute_face_vectors()
for i, item in enumerate(mesh_geocal.mesh.blocks):
    print(f"block {i} keys: {list(item.keys())}")
print(mesh_geocal.mesh.blocks[0]['bc'])

[Block 0] xc range: (-13.69490000, 14.69897015), yc range: (-14.45284450, 14.45210219)
[Block 1] xc range: (1.00027755, 14.70264261), yc range: (-0.12138114, 0.17834181)
[Block 0] volume range: (3.62171595e-07, 1.78960182e+00)
[Block 1] volume range: (3.85062192e-07, 1.38875202e-01)
block 0 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S2', 'S3', 'S4']
block 1 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S2', 'S3', 'S4']
[{'type': 2, 'source': (259, 1, 1, 1)}, {'source': (259, 259, 1, 49), 'type': -1, 'target': (1, 1, 1, 49), 'target_block': 1, 'transform': (1, 2)}, {'type': 4, 'source': (1, 259, 49, 49)}, {'source': (1, 1, 1, 49), 'type': -1, 'target': (10, 10, 1, 49), 'target_block': 1, 'transform': (1, 2)}]


# 添加虚网格

In [4]:
blocks = np.copy(mesh_geocal.mesh.blocks)
bd.crate_ghost_cells(blocks, config.GHOST_LAYER, config.N_C)
for block in blocks:
 for bc in block['bc']: 
    if 'ghost_cell' in bc:
        print("ghost_cell shape:", bc['ghost_cell'].shape)
for i, block in enumerate(blocks):
    print(f"Block {i} keys:", list(block.keys()))

ghost_cell shape: (258, 2, 4)
ghost_cell shape: (48, 2, 4)
ghost_cell shape: (258, 2, 4)
ghost_cell shape: (48, 2, 4)
ghost_cell shape: (9, 2, 4)
ghost_cell shape: (48, 2, 4)
ghost_cell shape: (9, 2, 4)
ghost_cell shape: (48, 2, 4)
Block 0 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S2', 'S3', 'S4']
Block 1 keys: ['shape', 'x', 'y', 'z', 'bc', 'xc', 'yc', 'volume', 'S1', 'S2', 'S3', 'S4']


# 整理为一个用于计算的列表

In [5]:
blocks_cal = tf.trans_list2numpy_2d(blocks, config.N_C)
for i, block in enumerate(blocks_cal):
    print(f"Block {i} keys:", list(block.keys()))
    print(f"Block {i} has {len(block['bc'])} boundary conditions.")
    for j, bc in enumerate(block['bc']):
        print(f"  BC {j} keys: {list(bc.keys())}")
        print(f"    → type: {bc.get('type', 'N/A')}")

Block 0 keys: ['geo', 'fluid', 'bc']
Block 0 has 4 boundary conditions.
  BC 0 keys: ['type', 'source', 'ghost_cell']
    → type: 2
  BC 1 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1
  BC 2 keys: ['type', 'source', 'ghost_cell']
    → type: 4
  BC 3 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1
Block 1 keys: ['geo', 'fluid', 'bc']
Block 1 has 4 boundary conditions.
  BC 0 keys: ['source', 'type', 'ghost_cell']
    → type: 2
  BC 1 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1
  BC 2 keys: ['source', 'type', 'ghost_cell']
    → type: 4
  BC 3 keys: ['source', 'type', 'target', 'target_block', 'transform', 'ghost_cell']
    → type: -1


# 初始化流场和边界条件

In [6]:
initial.intialization_from_farfield(blocks_cal)
print(blocks_cal[0]['fluid'][0, 0, 0:4])
print(blocks_cal[0]['bc'][0]['ghost_cell'][0, 0, 0:4])

[ 1.          0.99939083  0.0348995  20.34126984]
[ 1.          0.99939083  0.0348995  20.34126984]


# 迭代计算
## 边界条件处理(虚网格)

In [7]:
bd.boundary_farfeild(blocks_cal)
bd.boundary_wall_inviscid(blocks_cal)
bd.boundary_interface(blocks_cal)


print(blocks_cal[0]['fluid'][0, 0, 0:4])
print(blocks_cal[0]['geo'][0, 0, 3:5])
print(blocks_cal[0]['geo'][0, 0, 3:5] / np.linalg.norm(blocks_cal[0]['geo'][0, 0, 3:5]))
print(blocks_cal[0]['bc'][0]['ghost_cell'][0, 0, 0:4])

v_r = np.array((blocks_cal[0]['fluid'][26, 0, 1], blocks_cal[0]['fluid'][26, 0, 2]))
v_g = np.array((blocks_cal[0]['bc'][0]['ghost_cell'][26, 0, 1], blocks_cal[0]['bc'][0]['ghost_cell'][26, 0, 2]))
S = blocks_cal[0]['geo'][26, 0, 3:5] / np.linalg.norm(blocks_cal[0]['geo'][26, 0, 3:5])
flux = np.dot(v_r + v_g, S)
print(flux)

[ 1.          0.99939083  0.0348995  20.34126984]
[-6.73797273e-05  4.95439171e-04]
[-0.13475945  0.99087834]
[ 1.          0.97241301  0.23326582 20.34126984]
-5.551115123125783e-17


## 验证空间离散

In [8]:
from solver.flux import reconstruct_interface_state as re
from solver.flux import conflux_roe

u_stat = re(blocks_cal[0], [4,4], [1,1])
s = blocks_cal[0]['geo'][4,4, 5:7]
flux = conflux_roe(u_stat[:,0], u_stat[:,1], s)
print(flux)
u_stat = re(blocks_cal[0], [4,4], [1,-1])
s = blocks_cal[0]['geo'][4,4, 9:11]
flux = flux + conflux_roe(u_stat[:,0], u_stat[:,1], s)
print(flux)
u_stat = re(blocks_cal[0], [4,4], [2,1])
s = blocks_cal[0]['geo'][4,4, 7:9]
flux = flux + conflux_roe(u_stat[:,0], u_stat[:,1], s)
print(flux)
u_stat = re(blocks_cal[0], [4,4], [2,-1])
s = blocks_cal[0]['geo'][4,4, 3:5]
flux = flux + conflux_roe(u_stat[:,0], u_stat[:,1], s)
print(flux)

print(blocks_cal[0]['fluid'][4, 4])
print(blocks_cal[0]['fluid'][4, 5])
print(blocks_cal[0]['fluid'][5, 4])
print(blocks_cal[0]['fluid'][3, 4])
print(blocks_cal[0]['fluid'][4, 3])

geo = blocks_cal[0]['geo']
print("S1+S3 = ", geo[4,4,3:5] + geo[4,4,7:9])   # 下 + 上
print("S4+S2 = ", geo[4,4,5:7] + geo[4,4,9:11])  # 左 + 右

U = np.array([1.0, 1.0, 0.0, 2.0])
S = np.array([1.0, 0.0])  # 单位面矢量

flux = conflux_roe(U, U, S)
print(flux)
S = [0.0, 1.0]
flux = conflux_roe(U, U, S)
print(flux)

[-8.70424352e-01 -1.06498533e-02 -5.82345952e-03 -2.46136664e+01]
[-4.23759685e-02 -1.26762559e-03  2.73895015e-04 -1.19829822e+00]
[ 4.65481437e-01  3.71724296e-03 -6.69473740e-03  1.31627806e+01]
[ 5.26088685e-02 -4.33680869e-19  8.67361738e-19  1.48766189e+00]
[ 1.          0.99939083  0.0348995  20.34126984]
[ 1.          0.99939083  0.0348995  20.34126984]
[ 1.          0.99939083  0.0348995  20.34126984]
[ 1.          0.99939083  0.0348995  20.34126984]
[ 1.          0.99939083  0.0348995  20.34126984]
S1+S3 =  [ 1.42004439e-04 -3.51294417e-05]
S4+S2 =  [-1.42004439e-04  3.51294417e-05]
[1.  1.6 0.  2.6]
[-1.   0.   1.6 -2.6]
